In [1]:
from copy import deepcopy

import numpy as np
import open3d as o3d
import trimesh

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


simplify_vertex_clustering is not good for subdivide surface fitting, simplify_quadric_decimation is much better
notes that boundary_weight will influence the result of decimated mesh, inappropriate setting leads to holes in the mesh.

In [2]:
def subdivide_surface_fitting(decimated_mesh, target_mesh, iterations=1):
    subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_mesh, number_of_iterations=iterations)
    print(subdivided_mesh)
    subdivided_mesh.compute_vertex_normals()
    
    pcd_target = o3d.geometry.PointCloud()
    pcd_target.points = o3d.utility.Vector3dVector(target_mesh.vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_target)
    subdivided_vertices = np.array(subdivided_mesh.vertices)
    target_vertices = np.array(target_mesh.vertices)
    fitting_vertices = deepcopy(subdivided_vertices)
    
    for i in range(0, len(subdivided_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(subdivided_vertices[i], 1)
        fitting_vertices[i] = target_vertices[np.asarray(index)]
        
    subdivided_mesh.vertices = o3d.utility.Vector3dVector(fitting_vertices)
    return subdivided_mesh

In [14]:
deformed_meshes = []
all_meshes = o3d.geometry.TriangleMesh()
for i in range(5,20):
    mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\output/deformed_{i:03}.obj')
    mesh.compute_vertex_normals()
    deformed_meshes.append(mesh)
    all_meshes += mesh
o3d.visualization.draw_geometries([all_meshes])
print(all_meshes, all_meshes.has_vertex_normals())
reference_pcd = o3d.geometry.PointCloud()
reference_pcd.points = o3d.utility.Vector3dVector(all_meshes.vertices)
reference_pcd.normals = o3d.utility.Vector3dVector(all_meshes.vertex_normals)
print(reference_pcd, reference_pcd.has_normals())

TriangleMesh with 295815 points and 590681 triangles. True
PointCloud with 295815 points. True


In [15]:
GoF = 15
key = 7 # index of the key frame to fit 
fitting_meshes = []
for i in range(0,GoF):
    if i==key:
        continue
    decimated_mesh_i = o3d.geometry.TriangleMesh.simplify_quadric_decimation(deformed_meshes[i], 10000, boundary_weight= 8000)
    print(decimated_mesh_i)
    fitting_mesh_i = subdivide_surface_fitting(decimated_mesh_i, deformed_meshes[key], 1)
    print(fitting_mesh_i)
    fitting_mesh_i.compute_vertex_normals()
    fitting_meshes.append(fitting_mesh_i)
fitting_meshes.append(deformed_meshes[key])

TriangleMesh with 5038 points and 9999 triangles.
TriangleMesh with 20077 points and 39996 triangles.
TriangleMesh with 20077 points and 39996 triangles.
TriangleMesh with 5035 points and 9999 triangles.
TriangleMesh with 20072 points and 39996 triangles.
TriangleMesh with 20072 points and 39996 triangles.
TriangleMesh with 5038 points and 10000 triangles.
TriangleMesh with 20080 points and 40000 triangles.
TriangleMesh with 20080 points and 40000 triangles.
TriangleMesh with 5042 points and 9999 triangles.
TriangleMesh with 20088 points and 39996 triangles.
TriangleMesh with 20088 points and 39996 triangles.
TriangleMesh with 5041 points and 10000 triangles.
TriangleMesh with 20087 points and 40000 triangles.
TriangleMesh with 20087 points and 40000 triangles.
TriangleMesh with 5044 points and 10000 triangles.
TriangleMesh with 20093 points and 40000 triangles.
TriangleMesh with 20093 points and 40000 triangles.
TriangleMesh with 5042 points and 10000 triangles.
TriangleMesh with 2008

In [16]:
all_meshes = o3d.geometry.TriangleMesh()
for i in range(0,GoF):
    all_meshes += fitting_meshes[i]
print(all_meshes, all_meshes.has_vertex_normals())
reference_pcd = o3d.geometry.PointCloud()
reference_pcd.points = o3d.utility.Vector3dVector(all_meshes.vertices)
reference_pcd.normals = o3d.utility.Vector3dVector(all_meshes.vertex_normals)
print(reference_pcd, reference_pcd.has_normals())
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    pre_reference_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(reference_pcd, depth=9, linear_fit=True)
print(pre_reference_mesh)
pre_reference_mesh.compute_vertex_normals()

pre_reference_mesh.paint_uniform_color([0.7, 0.7, 0.7])
#o3d.visualization.draw_geometries([pre_reference_mesh])

reference_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(pre_reference_mesh, 40000, boundary_weight= 8000)
print(reference_mesh)
reference_mesh.compute_vertex_normals()

o3d.visualization.draw_geometries([reference_mesh])
o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh/reference_mesh_{GoF:02}.obj', reference_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)

TriangleMesh with 300838 points and 599329 triangles. True
PointCloud with 300838 points. True
run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 300838 / 18462
[Open3D DEBUG] #   Got kernel density: 0.0120001 (s), 437.273 (MB) / 497.426 (MB) / 651 (MB)
[Open3D DEBUG] #     Got normal field: 0.191 (s), 457.676 (MB) / 497.426 (MB) / 651 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 2.573483e-06 / 7.742016e-01
[Open3D DEBUG] #       Finalized tree: 0.161 (s), 502.918 (MB) / 502.918 (MB) / 651 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.347 (s), 484.707 (MB) / 502.918 (MB) / 651 (MB)
[Open3D DEBUG] #Set point constraints: 0.033 (s), 483.52 (MB) / 502.918 (MB) / 651 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 873398 / 998152 / 17
[Open3D DEBUG] Memory Usage: 483.520 MB
[Open3D DEBUG] # Linear system solved: 0.352 (s), 518.879 (MB) / 518.879 (MB) / 651 (MB)
[Open3D DEBUG] Got average: 0.00300002 (s), 497.621 (MB) / 518.879 (MB) / 651 (MB)
[Open

True

In [17]:
load_reference_mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh/reference_mesh_{GoF:02}.obj', enable_post_processing = False)
print(load_reference_mesh)

o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh/reference_mesh.obj_{GoF:02}', load_reference_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
load_reference_mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh/reference_mesh_{GoF:02}.obj', enable_post_processing = False)
print(load_reference_mesh)
reference_vertices = np.array(load_reference_mesh.vertices)
load_reference_mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([load_reference_mesh])
load_reference_mesh.paint_uniform_color([0.7, 0.7, 0.7])
print(reference_vertices)

TriangleMesh with 19993 points and 40000 triangles.
[Open3D WARNING] Write geometry::TriangleMesh failed: unknown file extension.
TriangleMesh with 19993 points and 40000 triangles.
[[ 0.0789528   0.89884597 -0.47032499]
 [ 0.0719497   0.88984197 -0.468503  ]
 [ 0.074003    0.90584898 -0.46656099]
 ...
 [-0.33023399  0.80180103  0.131146  ]
 [-0.33223501  0.80980498  0.129778  ]
 [-0.0125943   1.04575002  0.192404  ]]


In [18]:
load_reference_mesh_trimesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer/reference_mesh/reference_mesh_{GoF}.obj', enable_post_processing = False)

print(np.array(load_reference_mesh_trimesh.vertices).__len__())
decimated_reference_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(load_reference_mesh_trimesh, 10000, boundary_weight= 8000)
print(decimated_reference_mesh)

subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_reference_mesh, number_of_iterations=1)
print(subdivided_mesh)
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/decimated_reference_mesh.obj', decimated_reference_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
o3d.io.write_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/subdivided_reference_mesh.obj', subdivided_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)


19993
TriangleMesh with 4993 points and 10000 triangles.
TriangleMesh with 19993 points and 40000 triangles.


True

## this function helps avoid that the order of vertices changes, which is caused by open3d

In [9]:
def read_triangle_mesh_with_trimesh(avatar_name,enable_post_processing=False):
    # EDIT: next 4 lines replace to maintain order even in case of degenerate and non referenced
    # scene_patch = trimesh.load(avatar_name,process=enable_post_processing)
    if enable_post_processing:
        scene_patch = trimesh.load(avatar_name,process=True)
    else:
        scene_patch = trimesh.load(avatar_name,process=False,maintain_order=True) 
    mesh = o3d.geometry.TriangleMesh(
        o3d.utility.Vector3dVector(scene_patch.vertices),
        o3d.utility.Vector3iVector(scene_patch.faces)
    ) 
    if scene_patch.vertex_normals.size:
        mesh.vertex_normals = o3d.utility.Vector3dVector(scene_patch.vertex_normals.copy())
    if scene_patch.visual.defined:
        # either texture or vertex colors if no uvs present.
        if scene_patch.visual.kind == 'vertex':
            mesh.vertex_colors = o3d.utility.Vector3dVector(scene_patch.visual.vertex_colors[:,:3]/255) # no alpha channel support
        elif scene_patch.visual.kind == 'texture':
            uv = scene_patch.visual.uv
            if uv.shape[0] == scene_patch.vertices.shape[0]:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv[scene_patch.faces.flatten()])
            elif uv.shape[0] != scene_patch.faces.shape[0] * 3:
                assert False
            else:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv)
                if scene_patch.visual.material is not None and scene_patch.visual.material.image is not None:
                    if scene_patch.visual.material.image.mode == 'RGB':
                        mesh.textures = [o3d.geometry.Image(np.asarray(scene_patch.visual.material.image))]
                    else:
                        assert False
        else:
            assert False
    return mesh

## deal with reorder problem

try store a reindexer in color attribute, but Draco eliminates all color information when encoding, so this is not feasible for our project

In [ ]:
mesh = deepcopy(load_reference_mesh)
true_colors = np.array(mesh.vertex_colors,copy=True)
vertex_reindexer = np.arange(len(mesh.vertices))
mesh.vertex_colors = o3d.utility.Vector3dVector((vertex_reindexer / vertex_reindexer[-1]).reshape(-1,1)[:,[0,0,0]])
o3d.io.write_triangle_mesh(
    r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/order_reference_mesh.obj',mesh,
    write_ascii=False,write_vertex_normals = False, write_vertex_colors = True
)
hardened_mesh = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/order_reference_mesh.obj',enable_post_processing = False)
remaining_reordered_vertices = (np.asarray(hardened_mesh.vertex_colors)[:,0] * vertex_reindexer[-1] + 0.5).astype(int)
hardened_mesh.vertex_colors = o3d.utility.Vector3dVector(true_colors[remaining_reordered_vertices])
# reorder all additional data linked to the vertices using the remaining_reordered_vertices
# store the hardened_mesh using the final filename